In [1]:
import import_hack
import steps
from data_store import sqlite_table_datastore as sqlds
from data_store import sqlite_table_one_to_many_datastore as sqlotmds
from data_store import file_system_directory_datastore as fsds
from data_store import stream_ndarray_adapter_datastore as snads
from transformer import transformers as trs
%load_ext autoreload
%autoreload 2

In [2]:
sample_part=0.1
sqlite_db_path_in='ds_data\example_compute_siftid_sift_imgid'
sqlite_table_name_in='siftid_sift_imgid'
sqlite_db_path_out='ds_data\example_compute_siftid_sift_imgid_samples{0}'.format(sample_part)
sqlite_table_name_out='siftid_sift'

We can interpret table as One-to-One(id, itemid) or as One-to-Many(id, itemid, foreignid).
If we want to us it like One-to-One, use SQLiteTableDataStore. Here item = one sift.
If we want to us it like One-to-Many, use SQLiteTableOneToManyDataStore. In this case items would be grouped by foreignid and aggregated into ndarray. Here item = ndarray of sifts, e.g. ndarray with shape (312, 128)

In [3]:
ds_in = sqlds.SQLiteTableDataStore(sqlite_db_path_in, sqlite_table_name_in)
ds_out = sqlds.SQLiteTableDataStore(sqlite_db_path_out, sqlite_table_name_out)

In [4]:
with ds_in:
    total_sifts_count=ds_in.get_count()
    print(total_sifts_count)

826845


In [5]:
steps.sampling_step(ds_in, sample_part, ds_out)

Here we treat table as One-to-One, so only (id,item) were sampled, foreignid was not considered.

In [6]:
with ds_out:
    sampled_sifts_count=ds_out.get_count()
    print(sampled_sifts_count)
    print(total_sifts_count*sample_part)

82684
82684.5


In [7]:
with ds_out:
    items=ds_out.get_items_sorted_by_ids()
    for i in range(3):
        print(next(items).shape)

(128,)
(128,)
(128,)


In [6]:
import os
os.remove(sqlite_db_path)